In [12]:
import os
import pandas as pd

In [13]:
from modules.config import DATADIR
from modules.utils import load_json, print_groupby

## Merge factory contract data (transaction function call input data + contract creation from trace)

In [15]:
# Load factory contract addresses for each DAO framework
df_factories = pd.read_csv(os.path.join(DATADIR, 'framework_factory_contract_addresses.csv'), index_col=False)
df_factories.set_index('version', inplace=True)
df_factories['abis'] = None
df_factories['relevantParams'] = None

# Load abis for each DAO framework
for version, row in df_factories.iterrows():
    abis_full = load_json(os.path.join(DATADIR, f"trueblocks_framework_abis_{version}.json"))['data']
    abis = {}
    for abi in abis_full:
        abiDict = {k: abi[k] for k in ['name', 'type']}
        abis[abi['name']] = {v['name']: v['type'] for v in abi.get('inputs', {})}
    df_factories.at[version, 'abis'] = abis
    try:
        df_factories.at[version, 'relevantParams'] = abis[row['relevantABI']]
    except KeyError:
        df_factories.at[version, 'relevantParams'] = None

abiLookup = {i: r['relevantABI'] for i, r in df_factories.iterrows()}

df_factories

,framework,factoryAddress,relevantABI,abis,relevantParams
version,,,,,
aragon_0.8.1,aragon,0x7378ad1ba8f3c8e64bbb2a04473edd35846360f1,newTokenAndBaseInstance,"{'baseACL': {}, 'DeployEVMScriptRegistry': {'r...",None
aragon_0.8,aragon,0xb9da44c051c6cc9e04b7e0f95e95d69c6a6d8031,newTokenAndBaseInstance,"{'baseACL': {}, 'DeployEVMScriptRegistry': {'r...",None
aragon_0.7,aragon,0xc29f0599df12eb4cbe1a34354c4bac6d944071d1,newInstance,"{'baseACL': {}, 'DeployEVMScriptRegistry': {'r...",None
aragon_0.6,aragon,0x595b34c93aa2c2ba0a38daeede629a0dfbdcc559,newInstance,"{'baseACL': {}, 'DeployEVMScriptRegistry': {'r...",None
aragon_court,aragon,0xee4650cBe7a2B23701D416f58b41D8B76b617797,setConfig,"{'ModuleSet': {'id': 'bytes32', 'addr': 'addre...","{'_fromTermId': 'uint64', '_feeToken': 'addres..."
daostack_alchemy,daostack,0x5bcaf9bfd511166a6e0bcfe24b42f5932f43f4b7,forgeOrg,"{'setSchemes': {'_avatar': 'address', '_scheme...","{'_orgName': 'string', '_tokenName': 'string',..."
daohaus_molochv2.1,moloch,0x38064F40B20347d58b326E767791A6f79cdEddCe,summonMoloch,"{'SummonComplete': {'moloch': 'address', 'summ...","{'_summoner': 'address[]', '_approvedTokens': ..."
openlaw_tribute,tribute,0xf44e53e7474588494b3bec75898278050d99a8ce,createDAO,"{'DAOCreated': {'_address': 'address', '_name'...",None


In [ ]:
# Load transaction data from all factory contracts by framework version
df_transactions = pd.DataFrame()
files = [f for f in os.listdir(DATADIR) if f.startswith('trueblocks_export')]
for f in files:
    version = "_".join(os.path.splitext(f)[0].split('_')[2:])
    dataDir = load_json(os.path.join(DATADIR, f))
    dataList = [{'id': f"{version}_{k}",
                 'framework': df_factories.at[version, 'framework'],
                 'version': version,
                 'functionName': v.get('articulatedTx', {}).get('name'),
                 'functionInputs': v.get('articulatedTx', {}).get('inputs'),
                 'transactionHash': v.get('hash')}
                for k, v in dataDir.items() if int(k) != 0]
    df_tmp = pd.DataFrame(dataList)
    df_transactions = df_transactions.append(df_tmp, ignore_index=True)

gb = df_transactions.groupby('version')
print(gb['id'].count())

version
aragon_0.6            257
aragon_0.7            241
aragon_0.8            149
aragon_0.8.1          149
aragon_court          310
daohaus_molochv2.1    241
daostack_alchemy      125
openlaw_tribute       107
Name: id, dtype: int64


In [ ]:
# Load contract creation trace data from all factory contracts by framework version
files = [f for f in os.listdir(DATADIR) if f.startswith('trueblocks_trace')]
df_traces = pd.DataFrame()
for f in files:
    version = "_".join(os.path.splitext(f)[0].split('_')[2:])
    dataDir = load_json(os.path.join(DATADIR, f))
    # Get contracts created
    dataList = []
    for key, fullTrace in dataDir.items():
        if int(key) != 0:
            dataList.append({'id': f"{version}_{key}",
                            'version': version,
                            **fullTrace})
    df_tmp = pd.DataFrame(dataList)
    df_traces = df_traces.append(df_tmp, ignore_index=True)

gb = df_traces.groupby('version')
print(gb['id'].count())

version
aragon_0.6            291
aragon_0.7            241
aragon_0.8            149
aragon_0.8.1          149
daohaus_molochv2.1    256
daostack_alchemy      122
openlaw_tribute       107
Name: id, dtype: int64


In [ ]:
# Merge transaction data with contract creation data
df_data = pd.merge(df_transactions, df_traces, on ='id', how ="outer", suffixes=(None, "_y"))
df_data.drop(columns=['version_y'], inplace=True)
print(df_data.columns)

gb = df_data.groupby('version')
print(gb['id'].count())

df_data.to_csv(os.path.join(DATADIR, "trueblocks_factory_transactions_full.csv"), index=False)

Index(['id', 'framework', 'version', 'functionName', 'functionInputs',
       'transactionHash', 'contractsCreated'],
      dtype='object')
version
aragon_0.6            257
aragon_0.7            241
aragon_0.8            149
aragon_0.8.1          149
aragon_court          310
daohaus_molochv2.1    241
daostack_alchemy      125
openlaw_tribute       107
Name: id, dtype: int64


In [ ]:
#print(df_data['functionName'].unique())
#print(df_factories['relevantABI'].unique())

# Clean data
def has_relevant_function(row):
    try:
        result = abiLookup[row['version']].strip().lower() == row['functionName'].strip().lower()
    except (KeyError, AttributeError) as e:
        result = False
    return result

df_data['is_relevant'] = df_data.apply(has_relevant_function, axis=1)
df_data_filtered = df_data[df_data['is_relevant']]

gb = df_data_filtered.groupby('version')
print(gb['id'].count())

version
aragon_0.6            248
aragon_0.7            235
aragon_0.8            118
aragon_0.8.1          139
daohaus_molochv2.1    232
daostack_alchemy       46
openlaw_tribute        25
Name: id, dtype: int64


In [ ]:
df_data_filtered.to_csv(os.path.join(DATADIR, "trueblocks_factory_transactions.csv"), index=False)

## Data analysis

In [ ]:
df = df_data_filtered
gb = df.groupby(['framework', 'version'])

In [ ]:
for key, item in gb:
    print(key)
    g = pd.DataFrame(gb.get_group(key))
